# Binary classification models

In [1]:
import numpy as np
import pandas as pd 
# import matplotlib as mpl   
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy.signal import find_peaks, savgol_filter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from keras import layers, models
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, LeakyReLU, BatchNormalization, MaxPooling1D, Dropout
# from keras.utils import to_categorical
# import glob

2023-12-03 11:42:15.028824: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 11:42:15.056229: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 11:42:15.056835: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 11:42:15.570866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("../input/data.csv")

In [7]:
display(data.tail().T, data.shape)

,84,85,86,87,88
320,-6.045785,-31.838142,24.793552,-17.652415,10.166875
321,-2.643846,-15.583373,9.836824,-8.010407,17.069577
322,5.90005,-2.133733,-5.213078,0.189554,18.806734
323,26.189564,-0.542107,-17.161635,4.889334,10.689331
324,50.332134,-3.355853,-23.770711,10.708648,-1.810575
...,...,...,...,...,...
1648,-11.094865,-1.922447,15.928893,-6.342218,-4.904049
1649,-1.680172,3.625427,-1.459799,4.096149,14.514866
1650,11.265595,-4.314838,-8.896405,3.117341,9.982839
Diagnostic,VLP,VLP,VLP,VLP,VLP


(89, 1333)

## Data prepration

In [4]:
def preprocess_data(data):

    X = data.drop(['Diagnostic', 'COVID-19_Status'], axis=1)
    X.columns = X.columns.astype(int)
    y = data['COVID-19_Status']

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    return X_train, X_test, y_train_encoded, y_test_encoded

## Simple Neural Network

In [8]:
def simple_nn(data):

    X_train, X_test, y_train_encoded, y_test_encoded = preprocess_data(data)

    model = Sequential([
        Dense(2048, input_dim= X_train.shape[1], activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, y_train_encoded, epochs=5, verbose=1)
    loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
    print(f'Test loss: {loss:.2f}, Test accuracy: {accuracy*100:.2f}%')

    return model

In [9]:
simple_model = simple_nn(data)

Epoch 1/5
3/3 [==============================] - 0s 19ms/step - loss: 55.6347 - accuracy: 0.5493
Epoch 2/5
3/3 [==============================] - 0s 15ms/step - loss: 126.9982 - accuracy: 0.9296
Epoch 3/5
3/3 [==============================] - 0s 16ms/step - loss: 112.8934 - accuracy: 0.9437
Epoch 4/5
3/3 [==============================] - 0s 15ms/step - loss: 79.6113 - accuracy: 0.9437
Epoch 5/5
3/3 [==============================] - 0s 16ms/step - loss: 42.4668 - accuracy: 0.9437
Test loss: 27.62, Test accuracy: 94.44%


## CNN (Convolution Neural Network) model

In [10]:
def simple_cnn(data):

    X_train, X_test, y_train_encoded, y_test_encoded = preprocess_data(data)

    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
        layers.MaxPooling1D(2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    model.fit(X_train, y_train_encoded, epochs=10, validation_data=(X_test, y_test_encoded), verbose=0)

    test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
    print(f'Test Loss: {test_loss*100:.2f}% - Test Accuracy: {test_accuracy*100:.2f}%')

    return model

In [11]:
simple_cnn_model = simple_cnn(data)

Test Loss: 1132.91% - Test Accuracy: 77.78%


## SVM (Support Vector Machine) model

## Report